## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [17]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Victoria,HK,22.2855,114.1577,82.45,81,97,17.00,overcast clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,35.71,87,75,5.75,broken clouds
2,2,Dikson,RU,73.5069,80.5464,28.67,88,67,12.62,broken clouds
3,3,Arraial Do Cabo,BR,-22.9661,-42.0278,68.47,74,11,6.06,few clouds
4,4,Ponta Delgada,PT,37.7333,-25.6667,68.50,94,75,3.44,light intensity drizzle


In [18]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [19]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                     (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Victoria,HK,22.2855,114.1577,82.45,81,97,17.00,overcast clouds
9,9,Saint-Philippe,RE,-21.3585,55.7679,76.59,74,18,9.69,few clouds
11,11,Carnarvon,AU,-24.8667,113.6333,76.28,60,13,22.35,few clouds
21,21,Rikitea,PF,-23.1203,-134.9692,75.56,77,5,13.33,light rain
23,23,Pahrump,US,36.2083,-115.9839,71.26,12,0,1.99,clear sky
26,26,Bengkulu,ID,-3.8004,102.2655,84.61,65,56,6.44,broken clouds
27,27,Atuona,PF,-9.8000,-139.0333,77.61,79,20,8.46,light rain
29,29,Maryville,US,35.7565,-83.9705,73.60,79,0,0.00,clear sky
30,30,Alofi,NU,-19.0595,-169.9187,79.34,75,22,9.08,few clouds
33,33,Vaini,TO,-21.2000,-175.2000,77.16,88,75,5.75,broken clouds


In [20]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                260
City                   260
Country                260
Lat                    260
Lng                    260
Max Temp               260
Humidity               260
Cloudiness             260
Wind Speed             260
Current Description    260
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Victoria,HK,22.2855,114.1577,82.45,81,97,17.00,overcast clouds
9,9,Saint-Philippe,RE,-21.3585,55.7679,76.59,74,18,9.69,few clouds
11,11,Carnarvon,AU,-24.8667,113.6333,76.28,60,13,22.35,few clouds
21,21,Rikitea,PF,-23.1203,-134.9692,75.56,77,5,13.33,light rain
23,23,Pahrump,US,36.2083,-115.9839,71.26,12,0,1.99,clear sky


In [22]:
preferred_cities_df.count()

City_ID                260
City                   260
Country                260
Lat                    260
Lng                    260
Max Temp               260
Humidity               260
Cloudiness             260
Wind Speed             260
Current Description    260
dtype: int64

In [23]:
clean_travel_cities.columns

Index(['City_ID', 'City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Current Description'],
      dtype='object')

In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,82.45,overcast clouds,22.2855,114.1577,
9,Saint-Philippe,RE,76.59,few clouds,-21.3585,55.7679,
11,Carnarvon,AU,76.28,few clouds,-24.8667,113.6333,
21,Rikitea,PF,75.56,light rain,-23.1203,-134.9692,
23,Pahrump,US,71.26,clear sky,36.2083,-115.9839,
26,Bengkulu,ID,84.61,broken clouds,-3.8004,102.2655,
27,Atuona,PF,77.61,light rain,-9.8000,-139.0333,
29,Maryville,US,73.60,clear sky,35.7565,-83.9705,
30,Alofi,NU,79.34,few clouds,-19.0595,-169.9187,
33,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
   
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,82.45,overcast clouds,22.2855,114.1577,Mini Central
9,Saint-Philippe,RE,76.59,few clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
11,Carnarvon,AU,76.28,few clouds,-24.8667,113.6333,Hospitality Carnarvon
21,Rikitea,PF,75.56,light rain,-23.1203,-134.9692,People ThankYou
23,Pahrump,US,71.26,clear sky,36.2083,-115.9839,Best Western Pahrump Oasis
26,Bengkulu,ID,84.61,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
27,Atuona,PF,77.61,light rain,-9.8000,-139.0333,Villa Enata
29,Maryville,US,73.60,clear sky,35.7565,-83.9705,Holiday Inn Express & Suites Alcoa (Knoxville ...
30,Alofi,NU,79.34,few clouds,-19.0595,-169.9187,Taloa Heights
33,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,Keleti Beach Resort


In [27]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,82.45,overcast clouds,22.2855,114.1577,Mini Central
9,Saint-Philippe,RE,76.59,few clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
11,Carnarvon,AU,76.28,few clouds,-24.8667,113.6333,Hospitality Carnarvon
21,Rikitea,PF,75.56,light rain,-23.1203,-134.9692,People ThankYou
23,Pahrump,US,71.26,clear sky,36.2083,-115.9839,Best Western Pahrump Oasis
26,Bengkulu,ID,84.61,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
27,Atuona,PF,77.61,light rain,-9.8000,-139.0333,Villa Enata
29,Maryville,US,73.60,clear sky,35.7565,-83.9705,Holiday Inn Express & Suites Alcoa (Knoxville ...
30,Alofi,NU,79.34,few clouds,-19.0595,-169.9187,Taloa Heights
33,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,Keleti Beach Resort


In [29]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [30]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))